In [50]:
import requests
import time
import datetime
import pandas as pd

import hashlib
import hmac
import base64

import json

from pprint import pprint 

from pandas.io.json import json_normalize

import numpy
import logging

In [51]:
# Current time stamp
timestamp = int(time.time())

In [52]:
logging.basicConfig(level=logging.INFO, 
                    format=' %(asctime)s - %(levelname)s - %(message)s')

In [53]:
# Date
today_date = datetime.date.today() + datetime.timedelta(days=0)
yesterday_date = datetime.date.today() + datetime.timedelta(days=-1)
seven_days_before_date = datetime.date.today() + datetime.timedelta(days=-7)

In [54]:
# Shop Parameter
partner_id = 12155
shopid = 23070969
shop_key = 'de5e924b8ed680bc9b22a6c402058154340333ff05641a38069f2856a2f3e24e'

In [55]:
# get_auth_signature
def get_auth_signature(url, payload):
    # API Key
    shopee_secret = shop_key
    # Post Message
    post_message = url + '|' + json.dumps(payload)
    # HMAC-SHA256
    message = bytes(post_message, 'utf-8')
    secret = bytes(shopee_secret, 'utf-8')
    hash = hmac.new(secret, message, hashlib.sha256)
    auth_signature = hash.hexdigest()
    return auth_signature

In [56]:
# URL
url = 'https://partner.shopeemobile.com/api/v1/items/get'

In [57]:
# get all item
max_page = 200
pagination_entries_per_page = 100
i = 0

try:
    while i <= max_page:
        # start entry
        start_entry = i * pagination_entries_per_page

        # Parameter
        pagination_offset = start_entry

        # payload
        payload = {"partner_id":partner_id,
                   "shopid":shopid,
                   "timestamp":timestamp,
                   "pagination_offset":pagination_offset,
                   "pagination_entries_per_page":pagination_entries_per_page}
        # Auth Signature
        Auth_signature = get_auth_signature(url, payload)
        # HTML Header
        headers = {"Host": "partner.shopeemobile.com",
                    "Content-Type": "application/json",
                    "Content-Length": "89",
                    "Authorization": Auth_signature}
        # HTML Body
        payload = json.dumps(payload)
        # Post Request
        r = requests.post(url, data=payload, headers=headers)
        # 打印原始Json
        parsed = json.loads(r.text)
        # 漂亮打印Json
        item_list = json_normalize(parsed['items'])
        # 排除update_time
        selected_column = ['item_id', 'shopid', 'status']
        item_list = item_list[selected_column]

        # 合并
        if i == 0:
            all_item_list = item_list
        else:
            all_item_list = all_item_list.append(item_list)

        i += 1
        logging.info('Retrieving item from page ' + str(i))
except:
    pass

# 去重
logging.info('Remove duplicate records.')
all_item_list = all_item_list.drop_duplicates('item_id').reset_index()
logging.info('Retrieved item list.')
print(all_item_list.head())

 2018-05-11 12:58:10,927 - INFO - Retrieving item from page 1
 2018-05-11 12:58:11,189 - INFO - Retrieving item from page 2
 2018-05-11 12:58:11,369 - INFO - Remove duplicate records.
 2018-05-11 12:58:11,374 - INFO - Retrieved item list.


   index     item_id    shopid  status
0      0  1121446661  23070969  NORMAL
1      1  1121455576  23070969  NORMAL
2      2  1121423066  23070969  NORMAL
3      3   725349781  23070969  NORMAL
4      4   725351767  23070969  NORMAL


In [58]:
# get item detail
item_detail_url = 'https://partner.shopeemobile.com/api/v1/item/get'

In [59]:
item_detal_list = pd.DataFrame()

for key, item in all_item_list[:3].iterrows():
    try:
        item_id = item['item_id']
        logging.info('Now retrieving ' + str(item_id) + ' detail.')
        # payload
        payload = {"partner_id":partner_id,
                   "shopid":shopid,
                   "timestamp":timestamp,
                   "item_id": item_id}
        # Auth Signature
        Auth_signature = get_auth_signature(item_detail_url, payload)
        # HTML Header
        headers = {"Host": "partner.shopeemobile.com",
                    "Content-Type": "application/json",
                    "Content-Length": "89",
                    "Authorization": Auth_signature}
        # HTML Body
        payload = json.dumps(payload)
        # Post Request
        r = requests.post(item_detail_url, data=payload, headers=headers)
        # 打印原始Json
        parsed = json.loads(r.text)
        # 漂亮打印Json
        item_detail = json_normalize(parsed['item'])
        item_detal_list = item_detal_list.append(item_detail)
        # print(item_detail)
        logging.info('Retrieved ' + str(item_id) + ' detail.')
    except:
        pass

 2018-05-11 12:58:11,415 - INFO - Now retrieving 1121446661 detail.
 2018-05-11 12:58:11,702 - INFO - Retrieved 1121446661 detail.
 2018-05-11 12:58:11,704 - INFO - Now retrieving 1121455576 detail.
 2018-05-11 12:58:12,348 - INFO - Retrieved 1121455576 detail.
 2018-05-11 12:58:12,351 - INFO - Now retrieving 1121423066 detail.
 2018-05-11 12:58:12,633 - INFO - Retrieved 1121423066 detail.


In [60]:
item_detal_list.to_csv("D://item_detail_list.csv", sep=',')